In [6]:
import wfdb
import os
import pickle

In [7]:
def load_training_data(sample_path):
    record = wfdb.rdrecord(sample_path)
    fields = record.__dict__
    sig = fields['p_signal']
    length = fields['sig_len']
    fs = fields['fs']
    comments = fields['comments'][0]
    if comments == 'paroxysmal atrial fibrillation':
        result = 'AFp'
    elif comments == 'persistent atrial fibrillation':
        result = 'AFf'
    else:
        result = 'N'
        
    record_name = fields['record_name']

    return record_name, sig, length, fs, result

In [8]:
DATA_PATH = ['../../ECG Recordings/Training_set_I', '../../ECG Recordings/Training_set_II']
training_data_I = os.listdir(DATA_PATH[0])
training_data_II = os.listdir(DATA_PATH[1])

samples = [[], []]
for data in training_data_I:
    samples[0].append(data[:-4])
for data in training_data_II:
    samples[1].append(data[:-4])
samples = [list(set(samples[0])), list(set(samples[1]))]

In [9]:
training_data = []
for s in samples[0]:
    record_name, sig, length, fs, result = load_training_data(DATA_PATH[0] + '/' + s)
    data = {"record_name": record_name, "sig": sig, "sig_len": length, "fs": fs, "result": result}
    training_data.append(data)
    
for s in samples[1]:
    record_name, sig, length, fs, result = load_training_data(DATA_PATH[1] + '/' + s)
    data = {"record_name": record_name, "sig": sig, "sig_len": length, "fs": fs, "result": result}
    training_data.append(data)

In [11]:
with open('../data/data.pkl', 'wb') as file:
    pickle.dump(training_data, file)